<a href="https://colab.research.google.com/github/Shadrock/online-GIS-programming-course/blob/master/Week_1/30274_Analyzing_Airport_Data_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This lab was originally found at: http://opentechschool.github.io/python-data-intro/core/csv.html and uses data from https://openflights.org/data.html. This notebook contains the primary code used in the tutorial, and potential answers for the coding challenges, presented in the associated [Github repository for my programming in GIS course](https://github.com/Shadrock/online-GIS-programming-course/tree/master/Week_1). 

# **Analyzing Airport Data**
Before we get to the data, let's briefly touch on Reading and writing comma-separated data, or data from `.csv` files. Comma-separated values (CSV) is a way of expressing structured data in flat text files:

```
"Refugee_Camp_Name","Country","Population2006","Population2014"
"Kakuma","Kenya","90457","153959"
"Hagadera","Kenya","59185","106968"
"Adjumani","Uganda","54051","96926"
"Dagahaley","Kenya","39526","88486"
"Zaatari","Jordan","0","84773"
```


It’s also a commonly used format to get data in and out of programs like spreadsheet software, where the data are tabular. [Python comes with a CSV module](https://docs.python.org/3/library/csv.html) that provides one way to easily work with CSV-delimited data: Try importing the `Campsstats.csv` file, which contains the rows shown above, and then run the following code in Colab:

In [ ]:
# Upload local script to Colab here. 
from google.colab import files
uploaded = files.upload()

In [ ]:
import csv
f=open("Camp_stats.csv")
for row in csv.reader(f):
    print(row)

Each row is read as a list of strings representing the fields in the row.

## Why not use `.split()` or `.strip()`?

We already a learned another way to do this, we’ve learned `split(",")` to split each row of text into comma-delimited fields, and then `strip()` to take off the quote marks. (We did this with the survey in IDCE 302, if you recall!)

There are a few good reasons to use the CSV module here:
*   The csv module makes it clear what you’re doing to anyone reading your code.
*   The csv module is less likely to contain an error that splits some lines the wrong way.
*   The csv module has a lot of other features ([documented here](https://docs.python.org/3/library/csv.html)) that allow it to process differently formatted files, so you can easily update your program if the file format changes.



# **Reading Airport Data**
We’re going to do some processing of real-world data now, using freely available airline data sets from the [OpenFlights project](https://openflights.org/).

Visit the [OpenFlights data page](https://openflights.org/data.html) and download their airports data file - “airports.dat”. This is a file in CSV format, open it in a text editor if you want to have a look at it.

## Exercise 1

Can you use this file to print all of the airport names for a particular country (say, Australia or Russia)? To get you started, on the OpenFlights web page it shows that “Name” is the second field in each row of data. This means in the list of fields it will have index 1 (index 0 is the first field.)

As a starting point, here’s some code that prints the name of *every* airport:
```
import csv
f = open("airports.dat")
for row in csv.reader(f):
    print(row[1])
```

Be sure to start by uploading the airports.dat file. 
```
from google.colab import files
uploaded = files.upload()
````

### Answer

In [ ]:
# Upload local script to Colab here. 
from google.colab import files
uploaded = files.upload()

In [ ]:
import csv
f = open("airports.dat")
for row in csv.reader(f):
  if row[3] == "Australia" or row[3] == "Russia":
    print(row[1])

# **Creating an Airline Route Histogram**

We’re going to combine everything we’ve learned into a more complex problem to solve.

OpenFlights distribute databases for both airline locations and airline route details. You can download the routes database “routes.dat” from the OpenFlights data page. This database stores every unique flight route that OpenFlights knows about. Take a moment to look at the fields available in the routes data (listed on the OpenFlights page.)

By using both data sources, we can calculate how far each route travels and then plot a histogram showing the distribution of distances flown.

This a multiple stage problem:
*   Read the airports file (airports.dat) and build a dictionary mapping the unique airport ID to the geographical coordinates (latitude & longitude.) This allows you to look up the location of each airport by its ID.
*   Read the routes file (routes.dat) and get the IDs of the source and destination airports. Look up the latitude and longitude based on the ID. Using those coordinates, calculate the length of the route and append it to a list of all route lengths.
*   Plot a histogram based on the route lengths, to show the distribution of different flight distances.



## Exercise 2 - Reading the airport database
Write code to read through “airports.dat” and create a dictionary mapping from an airport ID key (use the numeric ID in the first field) to the geographic coordinates. You may want to create two dictionaries, one holding latitudes and one holding longitudes.

Look back at the OpenFlights data page to see the fields available in the airports.dat file. Hint: I ended up having to look at a copy of their data in their Github repo. 

### Answer 2


In [ ]:
latitudes = {}
longitudes = {}
f = open("airports.dat")
for row in csv.reader(f):
  airport_id = row[0]
  latitudes[airport_id] = float(row[6])
  longitudes[airport_id] = float(row[7])

# Running this won't display anything. If you're using a local editor or want to see the dictinoary use:
# print(latitudes)
# print(longitudes)
# If working in Colab or a similar Jupyter Notebook simply put the variable name in a cell and run it. 

In [ ]:
# Example of running a variable name in a cell 
latitudes

A few things to note in the code above: I used `float(row[6])`. which converted the string value to a floating point (ie fractional) number. We've learned this before, but let's review what's happening:

*   The CSV module returns the latitude as a string like “-6.081689”
*   You can convert this to an *integer* number (ie whole number) with `int(row[6])` but this would just be -6
Floating point numbers can store fractions, so we convert it to a floating point number with `float(row[6])` and get the full number -6.081689



## Exercise 3 - Route distances
Now that we have the lat/lon of each airport we can calculate the distance of each airline route.

Calculating geographic distances is a bit tricky because the earth is a sphere (actually, it's an oblate spheroid). The distance we measure is the “great circle distance”. We’re not going to implement our own great circle distance function in Python here, instead you can download a Python file with a `geo_distance()` function from here. Feel free to have a peek at it if you like, but don’t worry about completely understanding it at this stage. There are two ways you can use this function:
1. Place the file in the Notebook working directory and then import it as a Python module to use it (you can test it with the code below)


In [ ]:
# Code to import and check the geo_distance.py file to make sure it works.
geo_distance.distance(-37.814,144.963,52.519,13.406) # Melbourne to Berlin in km!

If you’re not using a Notebook, this code snippet doesn’t display anything. You’ll need to store the result of the distance function to a variable, then add a line with a print() statement to display the contents of the variable.

2. As an alternative to the import statement, you can also copy and paste the contents of the geo_distance.py file into an Notebook cell. Run the cell to define the distance function, and then use it in subsequent cells!

In [ ]:
# Copy of geo_distance.py
# Using the Haversine formula for geographic Great Circle Distance
# As per https://en.wikipedia.org/wiki/Haversine_formula

from math import cos,radians,sin,pow,asin,sqrt

def distance(lat1, long1, lat2, long2):
  radius = 6371 # radius of the earth in km, roughly https://en.wikipedia.org/wiki/Earth_radius
  # Lat,long are in degrees but we need radians
  lat1 = radians(lat1)
  lat2 = radians(lat2)
  long1 = radians(long1)
  long2 = radians(long2)

  dlat = lat2-lat1
  dlon = long2-long1

  a = pow(sin(dlat/2),2) + cos(lat1)*cos(lat2)*pow(sin(dlon/2),2)
  distance = 2 * radius * asin(sqrt(a))

  return distance


Once you have the `distance()` function working, can you write a program that reads all the airline routes from “routes.dat”, looks up the latitude and longitude of the source and destination airports, and builds a list of route distances?

When looking at the list of fields in the OpenFlights data documentation, remember that we used the “Unique OpenFlights identifier” fields for each airport when we made the dictionaries of latitudes and longitudes, not the multi-letter airport codes.

TIP: You might come across an error like “KeyError: \N” when you first run your program. This is another problem of ‘dirty data’, the “routes.dat” file contains some airports that aren’t listed in “airports.dat”. You can skip these routes by adding a test of the type `if airport in latitudes`.

### Answer 3

In [ ]:
# If you've previously uploaded the "airports.dat" file to this notebook you'll also need to add the "routes.dat" file to Colab here. 
from google.colab import files
uploaded = files.upload()

In [ ]:
distances = []
f = open("routes.dat")
for row in csv.reader(f):
  source_airport = row[3]
  dest_airport = row[5]
  if source_airport in latitudes and dest_airport in latitudes:
    source_lat = latitudes[source_airport]
    source_long = longitudes[source_airport]
    dest_lat = latitudes[dest_airport]
    dest_long = longitudes[dest_airport]
    distances.append(distance(source_lat,source_long,dest_lat,dest_long))

Once again, you can test the results of your program by typing the name of the output list (`distances` in this case) into a Notebook cell and running it or by adding `print()` statements to the end of the program to display the variable values.

In [ ]:
distances

## Exercise 4 - Histogram
Now we’re ready to create a histogram displaying the frequency of flights by distance.

Be sure to check out `plt.hist()`, which can do most of the work here. The first argument you supply will be the dataset (list of distances.)

The second argument (try starting with 100) is the number of bins to divide the histogram up into. You can increase this number to see more distinct bars and a more detailed picture, or reduce it to see a coarser picture. Try setting it to some other values and see what happens to the histogram plot.

The third argument, `facecolor`, sets the colour of the graph, “r” for red. There are a lot of ways to specify colours in matplotlib, all of which are [explained in the documentation](https://matplotlib.org/api/colors_api.html). All of the arguments that can be used with `hist()` can be found in the [matplotlib documentation](https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.hist).



### Answer 4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.hist(distances, 100, facecolor='r')
plt.xlabel("Distance (km)")
plt.ylabel("Number of flights")


### Congrats, you're done!
You’ve taken some real world data and graphed it in an informative way! Granted, the results aren’t earth shattering but you’re well on your way to understanding the techniques to perform other data analysis, and chart other data.